In [ ]:
#install.packages("RPostgreSQL",repos="http://cran.us.r-project.org/")
library(dplyr)
library(RSQLite)
library(nycflights13)
#library(RPostgreSQL)

In [ ]:
my_db <- src_sqlite("Data/my_db.sqlite", create = T)

flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list(
  c("year", "month", "day"), "carrier", "tailnum"))

In [ ]:
flights_sqlite <- tbl(nycflights13_sqlite(), "flights")

In [ ]:
flights_sqlite

In [ ]:
my_db

tbl(my_db, sql("SELECT * FROM flights"))

In [ ]:
select(flights_sqlite, year:day, dep_delay, arr_delay)

In [ ]:
filter(flights_sqlite, dep_delay > 240)

In [ ]:
arrange(flights_sqlite, year, month, day)

In [ ]:
mutate(flights_sqlite, speed = air_time / distance)

In [ ]:
summarise(flights_sqlite, delay = mean(dep_time))

In [ ]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, year, month, day, carrier)

In [ ]:
collect(c4)

In [ ]:
c4$query

In [ ]:
explain(c4)

In [ ]:
translate_sql(x)
translate_sql("x")

In [ ]:
translate_sql(x == 1 && (y < 2 || z > 3))

In [ ]:
translate_sql(x ^ 2 < 10)
translate_sql(x %% 2 == 10)

In [ ]:
translate_sql(1)
translate_sql(1L)
translate_sql(x %like% "ab*")

In [ ]:
by_tailnum <- group_by(flights_sqlite, tailnum)
delay <- summarise(by_tailnum,
  count = n(),
  dist = mean(distance),
  delay = mean(arr_delay)
)
(delay <- filter(delay, count > 20, dist < 2000))
(delay_local <- collect(delay))

In [ ]:
flights_postgres <- tbl(src_postgres("nycflights13"), "flights")

In [ ]:
bestworst <- daily %>% 
  select(flight, arr_delay) %>% 
  filter(arr_delay == min(arr_delay) || arr_delay == max(arr_delay))
bestworst$query

In [ ]:
ranked <- daily %>% 
  select(arr_delay) %>% 
  mutate(rank = rank(desc(arr_delay)))
ranked$query